# Step 4: 데이터셋 분할

통합된 데이터셋을 Train/Val/Test로 분할합니다.

## 왜 분할이 필요한가?
머신러닝에서 데이터를 세 부분으로 나누는 것은 필수입니다:
- **Train (70%)**: 모델 학습에 사용. 모델이 패턴을 배우는 데이터
- **Val (15%)**: 하이퍼파라미터 튜닝, Early Stopping 판단에 사용
- **Test (15%)**: 최종 성능 평가에 사용. 훈련 중 한 번도 보지 않은 데이터

## 분할 비율 선택 기준
- 70/15/15: 일반적인 비율 (데이터셋이 큰 경우)
- 80/10/10: 데이터셋이 작은 경우
- 60/20/20: 검증/테스트를 더 중요시하는 경우

## 랜덤 시드 (seed=42)
- 같은 시드를 사용하면 항상 같은 분할 결과가 나옴
- 재현성(reproducibility) 보장

## 입력/출력
- 입력: images/processed/merged/ (Step 3 결과)
- 출력:
  - images/train/images/, images/train/labels/
  - images/val/images/, images/val/labels/
  - images/test/images/, images/test/labels/

In [ ]:
import shutil
import random
from pathlib import Path

## 설정

In [ ]:
# 분할 비율
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

# 랜덤 시드 (재현성 보장)
seed = 42
random.seed(seed)

print(f"분할 비율: Train {train_ratio*100:.0f}% / Val {val_ratio*100:.0f}% / Test {test_ratio*100:.0f}%")
print(f"랜덤 시드: {seed}")

## 경로 설정

In [ ]:
# 프로젝트 루트 디렉토리
base_dir = Path.cwd().parent

# 입력 디렉토리
merged_dir = base_dir / 'images' / 'processed' / 'merged'
merged_images = merged_dir / 'images'
merged_labels = merged_dir / 'labels'

# 출력 디렉토리
output_base = base_dir / 'images'
splits = {
    'train': output_base / 'train',
    'val': output_base / 'val',
    'test': output_base / 'test'
}

print(f"입력 경로: {merged_dir}")

## 출력 디렉토리 생성

In [ ]:
for split_name, split_dir in splits.items():
    images_dir = split_dir / 'images'
    labels_dir = split_dir / 'labels'
    
    # 기존 파일 삭제
    if images_dir.exists():
        shutil.rmtree(images_dir)
    if labels_dir.exists():
        shutil.rmtree(labels_dir)
    
    # 새로 생성
    images_dir.mkdir(parents=True, exist_ok=True)
    labels_dir.mkdir(parents=True, exist_ok=True)

print("✅ 출력 디렉토리 생성 완료")

## 이미지-라벨 쌍 목록 생성

In [ ]:
# 라벨이 있는 이미지만 포함
all_images = []

for img_path in merged_images.glob('*'):
    if img_path.suffix.lower() in ['.jpg', '.jpeg', '.png']:
        label_path = merged_labels / (img_path.stem + '.txt')
        if label_path.exists():
            all_images.append(img_path.stem)

print(f"총 데이터: {len(all_images)}개")

## 랜덤 셔플 및 분할

In [ ]:
# 셔플
random.shuffle(all_images)

# 분할 인덱스 계산
total = len(all_images)
train_end = int(total * train_ratio)
val_end = train_end + int(total * val_ratio)

# 분할 수행
split_data = {
    'train': all_images[:train_end],
    'val': all_images[train_end:val_end],
    'test': all_images[val_end:]
}

for name, data in split_data.items():
    print(f"  {name}: {len(data)}개")

## 파일 복사

In [ ]:
print("\n파일 복사 중...")

for split_name, file_stems in split_data.items():
    split_dir = splits[split_name]
    images_dir = split_dir / 'images'
    labels_dir = split_dir / 'labels'
    
    for stem in file_stems:
        # 이미지 복사
        for ext in ['.jpg', '.jpeg', '.png']:
            src_img = merged_images / (stem + ext)
            if src_img.exists():
                shutil.copy(src_img, images_dir / src_img.name)
                break
        
        # 라벨 복사
        src_label = merged_labels / (stem + '.txt')
        if src_label.exists():
            shutil.copy(src_label, labels_dir / src_label.name)
    
    print(f"  {split_name}: {len(file_stems)}개 완료")

## 결과 확인

In [ ]:
print()
print("=" * 50)
print("✅ Step 4: 분할 완료!")
print("=" * 50)
print(f"   - Train: {len(split_data['train'])}개 ({len(split_data['train'])*100//total}%)")
print(f"   - Val: {len(split_data['val'])}개 ({len(split_data['val'])*100//total}%)")
print(f"   - Test: {len(split_data['test'])}개 ({len(split_data['test'])*100//total}%)")